<a href="https://colab.research.google.com/github/Simon-delalande/City-name-generator/blob/main/French_city_name_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model, inspired by the coursera's ..., aims to generate french cities names. 

In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.0 MB/s 


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import unidecode
import re
from keras.layers import LSTM, Dense


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/fr.csv')
cities= df['city']
df.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Paris,48.8566,2.3522,France,FR,Île-de-France,primary,11020000.0,2148271.0
1,Nice,43.7034,7.2663,France,FR,Provence-Alpes-Côte d’Azur,minor,1006402.0,341032.0
2,Toulouse,43.6045,1.4440,France,FR,Occitanie,admin,968638.0,486828.0
3,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,admin,870018.0,868277.0
4,Rennes,48.1147,-1.6794,France,FR,Bretagne,admin,727357.0,217728.0


In [ ]:
cities= cities.apply(lambda x:  re.sub(r'[()]', '', x))
cities= cities.apply(lambda x: list(unidecode.unidecode(x.lower()))+ ['stop'])
cities.head()

0                [p, a, r, i, s, stop]
1                   [n, i, c, e, stop]
2       [t, o, u, l, o, u, s, e, stop]
3    [m, a, r, s, e, i, l, l, e, stop]
4             [r, e, n, n, e, s, stop]
Name: city, dtype: object

In [ ]:
alphabet= "abcdeéèàfghijklmnopqrstuvwxyz -'"
alphabet= list(alphabet)
alph_size= len(alphabet)+1
alphabet += ['stop']
alph_to_ix= {ch: i for i, ch in enumerate(alphabet)}
ix_to_alph= {i: ch for i, ch in enumerate(alphabet)}

In [ ]:
def one_hot_encode(city):
  n= len(city)
  output= np.zeros((1, n, alph_size))
  for i in range(n):
    letter= city[i]
    id= alph_to_ix[letter]
    output[0,i, id]=1
  return(output)

In [ ]:
X= cities.apply(one_hot_encode)

In [ ]:
def remove_first_letter(x):
  return(x[:,1:,])


In [ ]:
remove_first_letter(X[0])

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1.]]])

In [ ]:
y= remove_first_letter(X[0])
for i in range(5):
  id= np.argmax(y[:,i,:], axis= -1)
  print(ix_to_alph[id[0]])

a
r
i
s
stop


In [ ]:
Y= np.zeros((df.shape[0]))
Y= np.array(Y, dtype= object)
for i in range(df.shape[0]):
  Y[i]= remove_first_letter(X[i])

In [ ]:
Y[0].shape


(1, 5, 33)

LSTM cell that inputs the previous state and a letter, and returns an updated state and a probability distribution over the next letter

In [ ]:
class generator(tf.keras.Model):
  def __init__(self, hidden_units):
    super(generator, self).__init__()
    self.units= hidden_units
  
  def build(self, input_shape=alph_size):
    self.LSTM= LSTM(units= self.units, return_sequences= True)
    self.dense= Dense(units= alph_size, activation= 'softmax')

  def call(self, inputs):
    # inputs.shape= (1, len_word, 27)
    len_word= inputs.shape[1]
    x= self.LSTM(inputs)
    x= self.dense(x)
    return (x)




In [ ]:
def loss(x,y):
  len_word= x.shape[1]
  ent= tf.keras.losses.CategoricalCrossentropy(axis=-1)
  return(ent(y, x[:,:len_word-1,:]))

In [ ]:
def train(model, X, Y, batch_size, epochs):
  tf.compat.v1.enable_eager_execution()
  optimizer = tf.keras.optimizers.Adam()
  len_set= X.shape[0]
  num_batches= len_set//batch_size
  rest= len_set% batch_size

  for n in range(epochs): 

    for i in range(num_batches):

      with tf.GradientTape() as tape:
        loss_value= 0
        for k in range(batch_size):

          input= X[i*batch_size+ k]
          y= Y[i*batch_size + k]
          pred= model(input)
          loss_value += loss(pred, y)

      gradients = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    for k in range(rest):

      loss_value= 0
      with tf.GradientTape() as tape:

          input= X[num_batches*batch_size+ k ]
          y= Y[num_batches*batch_size + k]
          pred= model(input)
          loss_value += loss(pred, y)
      print(loss_value)
      gradients = tape.gradient(loss_value, model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, model.trainable_weights))



In [ ]:
X= X.to_numpy()

AttributeError: ignored

In [ ]:
model= generator(64)
train(model, X, Y,32, 5)

tf.Tensor(2.584978, shape=(), dtype=float32)
tf.Tensor(3.1017263, shape=(), dtype=float32)
tf.Tensor(2.4113111, shape=(), dtype=float32)
tf.Tensor(2.310971, shape=(), dtype=float32)
tf.Tensor(2.533385, shape=(), dtype=float32)
tf.Tensor(2.898724, shape=(), dtype=float32)
tf.Tensor(2.6856856, shape=(), dtype=float32)
tf.Tensor(2.3497105, shape=(), dtype=float32)
tf.Tensor(2.3013597, shape=(), dtype=float32)
tf.Tensor(2.9127991, shape=(), dtype=float32)
tf.Tensor(2.490758, shape=(), dtype=float32)
tf.Tensor(2.484972, shape=(), dtype=float32)
tf.Tensor(2.4159873, shape=(), dtype=float32)
tf.Tensor(2.5975804, shape=(), dtype=float32)
tf.Tensor(2.5331569, shape=(), dtype=float32)
tf.Tensor(2.547188, shape=(), dtype=float32)
tf.Tensor(2.8142648, shape=(), dtype=float32)
tf.Tensor(2.6773534, shape=(), dtype=float32)
tf.Tensor(2.6130192, shape=(), dtype=float32)
tf.Tensor(2.0411391, shape=(), dtype=float32)
tf.Tensor(2.7483623, shape=(), dtype=float32)
tf.Tensor(1.9260263, shape=(), dtype=floa

KeyboardInterrupt: ignored

In [ ]:
z=np.zeros((1, 3, alph_size))
l= model(z)[0,-1,:]
np.sum(l)

1.0000001

In [ ]:
def sample(prob_dist):
  prob_dist /= prob_dist.sum()
  id= np.random.choice(np.arange(0, alph_size), p=prob_dist)
  return(id)


In [ ]:
def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string

In [ ]:
def generate_word(model):

  out_id= 0
  id_first= np.random.randint(0, alph_size)
  id= [id_first]

  one_hot = np.zeros((1,1, alph_size))
  one_hot[:,:,id_first]= 1

  while out_id != alph_to_ix['stop']:
    out= model(one_hot)
    last= out[0,-1,:].numpy()
    out_id= sample(last)
    id.append(out_id)
    new_arr= np.zeros((1, 1, alph_size))
    new_arr[:,:, out_id]=1
    one_hot= np.append(one_hot, new_arr, axis=1)

  name=''
  for i in id:
    name += ix_to_alph[i]
  name= remove_suffix(name, 'stop')
  return(name)


In [ ]:
for _ in range(100):
 print(generate_word(model))

zane
xy
pilenereuven
àyeqsaut-meaure
'umevil
jaroug
petuille
yleugen
khesy-care-autess
windaclernee
dertas
àeifn
'ourqga
llolqulac
feons
fiacavo
vwilelilds-eus-dap
vieroives
pborzogain
woin
a ciint-geyozs
zat-bagney
dares
rzat
ray
w ovalronhdes
ger ponresut
haries-loui
àrmaintes
éic
pejsaut-srtehelce
yhurons
 qgauiere-lera-bome
lo hheryne-uulasme
solbainnec
'irtdantrebofl-ue-frit
é obbaille
inat
stopablerers-cr-madepe
y-bouthred
t-acns-lchaigna-ds-crtor
'plegearen
zourpt-marsont
ruis
cillec
kussant-pu-blane
dnhey
'ouriceu
ixou
gnoser
bonges
éignac-guen
ulennes
 xgconnine
vendav
tèevele
isravvinle
vousunt
lheiniez
vounin
fbeys
dhout-lorilsa-
re-smimouc-seirtant
borsingurilee
zoyy
rnvaux
spartevis-nera
cenlamguve-degyrer
khaign
xdguvey
rercas
stopn
sailglre
ebollesne
nonbapdiu
qvironiy
dondgere
demeshourdit
o des
èvdain-sif-beretied
orintagx
qirzerialls
heinseffoms
iecres
qopevon-tricalit
walgan
taigneul
sransmairx
zamionh
xastetraf-cmerunag
jmekltessles
cedonde
saatzes
roimnis
marint
ée